# Perform Longitudinal Analysis

## Plot diversity metrics over time

We can first plot the diversity metrics over time to observe when abrupt changes in alpha diversity may have taken place.

In [ ]:
%%bash
# Volatility Analysis - Faith PD
qiime longitudinal volatility \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-file AR-core-metrics-results/faith_pd_vector.qza \
    --p-default-metric faith_pd \
    --p-default-group-column Pipe_Material \
    --p-state-column Months_Since_Start \
    --p-individual-id-column Sample_Identifier \
    --o-visualization longitudinal/AR-faith-pd-volatility.qzv

# Volatility Analysis - Shannon
qiime longitudinal volatility \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-file AR-core-metrics-results/shannon_vector.qza \
    --p-default-metric shannon \
    --p-default-group-column Pipe_Material \
    --p-state-column Months_Since_Start \
    --p-individual-id-column Sample_Identifier \
    --o-visualization longitudinal/AR-shannon-volatility.qzv

# Volatility Analysis - Simpson
qiime longitudinal volatility \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-file AR-core-metrics-results/simpson_vector.qza \
    --p-default-metric simpson \
    --p-default-group-column Pipe_Material \
    --p-state-column Months_Since_Start \
    --p-individual-id-column Sample_Identifier \
    --o-visualization longitudinal/AR-simpson-volatility.qzv

## Calculate the magnitude of change in alpha-diversity between successive timepoints

In [ ]:
%%bash
# Faith PD first differences
qiime longitudinal first-differences \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-file AR-core-metrics-results/faith_pd_vector.qza \
    --p-state-column Months_Since_Start \
    --p-metric faith_pd \
    --p-individual-id-column Sample_Identifier \
    --p-replicate-handling random \
    --o-first-differences longitudinal/faith-pd-first-differences.qza

# shannon first differences
qiime longitudinal first-differences \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-file AR-core-metrics-results/shannon_vector.qza \
    --p-state-column Months_Since_Start \
    --p-metric shannon \
    --p-individual-id-column Sample_Identifier \
    --p-replicate-handling random \
    --o-first-differences longitudinal/shannon-first-differences.qza

# simpson first differences
qiime longitudinal first-differences \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-file AR-core-metrics-results/simpson_vector.qza \
    --p-state-column Months_Since_Start \
    --p-metric simpson \
    --p-individual-id-column Sample_Identifier \
    --p-replicate-handling random \
    --o-first-differences longitudinal/simpson-first-differences.qza

### Visualize these results with the volatility visualizer

In [ ]:
%%bash
#faith-pd
qiime longitudinal volatility \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-file longitudinal/faith-pd-first-differences.qza \
    --p-default-metric Difference \
    --p-default-group-column Pipe_Material \
    --p-state-column Months_Since_Start \
    --p-individual-id-column Sample_Identifier \
    --o-visualization longitudinal/faith-pd-first-differences.qzv

# Shannon
qiime longitudinal volatility \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-file longitudinal/shannon-first-differences.qza \
    --p-default-metric Difference \
    --p-default-group-column Pipe_Material \
    --p-state-column Months_Since_Start \
    --p-individual-id-column Sample_Identifier \
    --o-visualization longitudinal/shannon-first-differences.qzv

# Simpson
qiime longitudinal volatility \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-file longitudinal/simpson-first-differences.qza \
    --p-default-metric Difference \
    --p-default-group-column Pipe_Material \
    --p-state-column Months_Since_Start \
    --p-individual-id-column Sample_Identifier \
    --o-visualization longitudinal/simpson-first-differences.qzv

## Calculate Beta Diversity Distance between Succesive Pairs

In [ ]:
%%bash
qiime longitudinal first-distances \
    --i-distance-matrix AR-core-metrics-results/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file sample-metadata.tsv \
    --p-state-column Months_Since_Start \
    --p-individual-id-column Sample_Identifier \
    --p-replicate-handling random \
    --o-first-distances longitudinal/first-distances.qza

# visualize the results
qiime longitudinal volatility \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-file longitudinal/first-distances.qza \
    --p-default-metric Distance \
    --p-default-group-column Pipe_Material \
    --p-state-column Months_Since_Start \
    --p-individual-id-column Sample_Identifier \
    --o-visualization longitudinal/first-distances.qzv

## Track rate of change in beta diversity from the first month

In [ ]:
%%bash
qiime longitudinal first-distances \
    --i-distance-matrix AR-core-metrics-results/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file sample-metadata.tsv \
    --p-state-column Months_Since_Start \
    --p-individual-id-column Sample_Identifier \
    --p-replicate-handling random \
    --p-baseline 2 \
    --o-first-distances longitudinal/first-distances-baseline-month2.qza

# visualize the results
qiime longitudinal volatility \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-file longitudinal/first-distances-baseline-month2.qza \
    --p-default-metric Distance \
    --p-default-group-column Pipe_Material \
    --p-state-column Months_Since_Start \
    --p-individual-id-column Sample_Identifier \
    --o-visualization longitudinal/first-distances-baseline-month2.qzv

## Handoff these results to R for plotting and significance testing in R

see scripts in /bin/scripts/ for plotting beta diversity and adonis PERMANOVA analysis (`nmit-adonis-test.R`).

## Test for differentially abundant features - Longitudinal ANCOM

longitudinal diiferential abundance testing was performed in R using the script `longitudinal_ANCOM.R`. Ths

### Alternatively, split into single pipe materials tables and perform significance testing

Instead of doing a two-factor test (which is not allowed in QIIME2) we can split the table by pipe material and then test for significance. First, create a newcategorical variable to test on - Start_Cal, the Months_Since_Start-Calendar_Month. Then filter any data from the table that has only one measurement for any given Start_Cal. Then split by Pipe_Material, recalculate core-metrics, and then significance test.

In [ ]:
%%bash
# Remove time points with only a single sample - these can't be permuted
qiime feature-table filter-samples \
    --i-table AR-filtered-table.qza \
    --m-metadata-file sample-metadata.tsv \
    --p-where "Start_Cal NOT IN ('12-Jun', '13-Jul', '14-Aug', '15-Sep', '16-Oct')" \
    --o-filtered-table AR-complete-samples-table.qza

# Now split into Pipe_Material Tables
qiime feature-table filter-samples \
    --i-table AR-complete-samples-table.qza \
    --m-metadata-file sample-metadata.tsv \
    --p-where "Pipe_Material = 'Cast Iron'" \
    --o-filtered-table cast-iron-table2.qza
    
qiime feature-table filter-samples \
    --i-table AR-complete-samples-table.qza \
    --m-metadata-file sample-metadata.tsv \
    --p-where "Pipe_Material = 'Cement'" \
    --o-filtered-table cement-table2.qza

# Calculate core-metrics for each
qiime diversity core-metrics-phylogenetic \
    --i-table cast-iron-table2.qza \
    --i-phylogeny rooted-tree.qza \
    --p-sampling-depth 34585 \
    --m-metadata-file sample-metadata.tsv \
    --output-dir cast-iron-core-metrics-results2
    
qiime diversity core-metrics-phylogenetic \
    --i-table cement-table2.qza \
    --i-phylogeny rooted-tree.qza \
    --p-sampling-depth 34585 \
    --m-metadata-file sample-metadata.tsv \
    --output-dir cement-core-metrics-results2

# Perform One-Way PERMANOVA on Start_Cal for each
qiime diversity beta-group-significance \
    --i-distance-matrix cast-iron-core-metrics-results2/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-column Start_Cal \
    --p-pairwise \
    --o-visualization cast-iron-core-metrics-results2/uUnifrac-Start_Cal-permanova.qzv
 
qiime diversity beta-group-significance \
   --i-distance-matrix cement-core-metrics-results2/unweighted_unifrac_distance_matrix.qza \
   --m-metadata-file sample-metadata.tsv \
   --m-metadata-column Start_Cal \
   --p-pairwise \
   --o-visualization cement-core-metrics-results2/uUnifrac-Start_Cal-permanova.qzv
 
# How about Season?
qiime diversity beta-group-significance \
    --i-distance-matrix cast-iron-core-metrics-results2/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file sample-metadata.tsv \
    --m-metadata-column Season \
    --p-pairwise \
    --o-visualization cast-iron-core-metrics-results2/uUnifrac-Season-permanova.qzv
 
qiime diversity beta-group-significance \
   --i-distance-matrix cement-core-metrics-results2/unweighted_unifrac_distance_matrix.qza \
   --m-metadata-file sample-metadata.tsv \
   --m-metadata-column Season \
   --p-pairwise \
   --o-visualization cement-core-metrics-results2/uUnifrac-Season-permanova.qzv